<a href="https://colab.research.google.com/github/JS-Choi513/Predicting-the-Housing-Price-Index-in-Changwon-City-Using-the-LSTM-Model/blob/main/Multi_step_LSTM_uhichang_6month_lagacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import backend as K
from google.colab import files
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
#!pip install pyyaml h5py


In [ ]:
uploaded = files.upload() 
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn]))) 
data = pd.read_csv('/content/integrated_data.csv')
data['new_Date'] = pd.to_datetime(data['date'])
print(data.head())
print('\n')
print(data.info())
print('\n')
#첫번째 컬럼을 날짜형식 컬럼으로 변경 
print(type(data['new_Date'][0]))
data.drop('date', axis = 1, inplace=True)
data.set_index('new_Date', inplace=True)
print(data.head())
print('\n')
print(data.info())

In [ ]:
#data['house_loan'].plot()
plt.title("household_loan")
plt.plot(data['house_loan'])
plt.show()
plt.title("consuming_price_idx")
plt.plot(data['consumer_price'])
plt.show()
plt.title("building_permitted_area")
plt.plot(data['building_space'])
plt.show()
plt.title("Interest_rate")
plt.plot(data['interest_idx'])
plt.show()
plt.title("housing price_idx")
plt.plot(data['housing_price'])
plt.show()


In [ ]:
data.head()
economic_stat = ['consumer_price','house_loan','building_space','interest_idx','housing_price']
features = data[economic_stat]
print(features.head(156))
print(features.values)
features.plot(subplots=True)
plt.show()

In [ ]:
#훈련데이터 분할 
#2008-01-01 ~ 2018-01-01 120개월을훈련데이터로 분할 
# 각 데이터 원시값-평균/표준편차 => 데이터셋 표준화 
# 156 = 
TRAIN_SPLIT =120
tf.random.set_seed(13)
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)#각 열에 대한120인덱스 까지의  평균값 
data_std = dataset[:TRAIN_SPLIT].std(axis=0)# 각 열에 대한 표준편차차
print(data_std)
dataset = (dataset-data_mean)/data_std
print(dataset)
print(dataset.shape)

In [ ]:
#신경망 모델 훈련에 사용한 특정시간 윈도우 데이터 확보를 위한 메소드 
#history: 과거데이터 크기
#target size 모델이 미래를 예측할 크기 
# step값이 주어지면 step에 맞게 데이터를 샘플링 

def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []          
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        print("dataset len %d",len(dataset))
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index+1):
        indices = range(i - history_size, i, step) 
        print(indices)
        data.append(dataset[indices])#
  
        if single_step:
          labels.append(target[i + target_size])
          print(label)
          print('타겟 데이터')
          print(target[i + target_size])
        else:
          labels.append(target[i:i + target_size])
                
    return np.array(data), np.array(labels)

'''
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []          
    labels = []
    start_index = start_index + history_size
    count = 26
    if end_index is None:
        print("dataset len %d",len(dataset))
        end_index = len(dataset) - target_size # -target size는 마지막에 라벨데이터로 쓸 데이터가 없기때문에 사용
        
    for i in range(start_index, end_index+1):
        indices = range(i - history_size, i, step) 
        
        if count < 125:# 각 광역시별 마지막 슬라이딩 윈도우의 라벨값이 
                      # 다음 광역시의 데이터로 들어가는것을 방지하기 위해
                      # 151까지 슬라이딩 윈도우를 생성한 
                      # 마지막 윈도우 (139 <= x < 151) +라벨 데이터: 151< y <=157 6개월 라벨 
          print(indices)
          data.append(dataset[indices])#
          labels.append(target[i:i + target_size])
        if count > 180: # 광역시 데이터가 겹치는 구간을 버리기 위함 슬라이딩 윈도우 끝부분이 150 이후부터 라벨데이터가 다음 광역시 값으로 들어감 따라서 167
          count = 26    # 따라서 슬라이딩 윈도우의 시작이 다음 광역시의 첫 부분이 될 때 까지 버림 
        count=count+1
        
    return np.array(data), np.array(labels)
     '''

In [ ]:
past_history = 26

# 13이나 39윈도우를 구성하면 각 광역시 마다독립적으로 윈도우 구성가능 
# 2008-01-01 ~ 2019-12-01  
# 13년치 *12 = 156행 * 광역시 7개 = 1092개 데이터 
# 서울 부산 인천 대구 광주 대전 울산 
# 156 약수 1 2 3 4 6 12 13 26 39 52 78 156
#
future_target = 6
STEP = 1
x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 4], 0, TRAIN_SPLIT, past_history, future_target, STEP)#0 ~ 30000
print('-----------------------------------')

# 예측해야할 데이터셋  11개 윈도우 120 ~ 154
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 4], TRAIN_SPLIT, None, past_history, future_target, STEP)#30000 ~ end

print('Single window of past history : {}'.format(x_train_multi[0].shape))
print(y_train_multi)
print('================================================================')
print('\n Target temperature to predict : {}'.format(y_val_multi[0].shape))
print(x_val_multi)
print('y')
print(y_val_multi)
# 처음부터 약2년까지의  3개월 단위 표준화 데이터셋 5개의 컬럼, 8개 행.

# 훈련데이터 차원 130,8,5..?

In [ ]:
base_model = tf.keras.models.load_model('/content/drive/MyDrive/6month_transfer_model2')
base_model.summary()
base_model.trainable = True
plot_model(base_model,to_file='/content/model.png',show_shapes=True)


In [ ]:
BATCH_SIZE = 256##144 #128#256
#Batch는 일괄 처리되는 작업의 양이다. 위에서 설명된 data_size를 한번에 처리하는 갯수를 의미하며, 
#batch크기에 의해 weight 변화가 일어나며 batch크기 단위로 data loading 함수도 구현이 가능하다. 
#주의할점은 data_size/batch_size일때 나머지가 없어야 한다.
tf.random.set_seed(13)
np.random.seed(13)
BUFFER_SIZE =  4096#데이터 14000
EVALUATION_INTERVAL = 28
EPOCHS = 720
# 410 0.00005
# 28/28 loss: 0.0022 - val_loss: 0.0359 700 -> 0.000001
# 입력된 텐서로부터 slices를 생성한다. 
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
# 입력된 텐서로부터 slices를 생성한다. 무기한 반복
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
# 데이터 캐싱, 훈련데이터 셔플 
val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi)) #11,9,5 11,0


val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()
print(x_train_multi.shape)
print(x_val_multi.shape)
print("----------")
print(x_train_multi.shape[-2:])
def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true)))


base_model = tf.keras.models.load_model('/content/drive/MyDrive/6month_transfer_model')
base_model.summary()
base_model.trainable = True
plot_model(base_model,to_file='/content/model.png',show_shapes=True)

multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.Dense(6,input_shape=base_model.output_shape[1:]))
multi_step_model.add(tf.keras.layers.Dense(6))
base_model.add(multi_step_model)
base_model.summary()


plot_model(base_model,to_file='/content/model2.png',show_shapes=True)
base_model.compile(optimizer=tf.keras.optimizers.Adam(0.000003), loss = 'mse')# 에러 절댓값 평균ingle_step_model.add(tf.keras.layers.Dense(1))
        #0.000005 0.0\360 580
import datetime
log_dir = "/content/log" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
multi_step_history = base_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=1)
#steps_per_epoch [트레이닝데이터수/배치사이즈]
#validation_steps validation data수/배치사이즈
base_model.summary()

 #0s 7ms/step - loss: 0.0069 - val_loss: 0.0349
print(multi_step_history)
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    
    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()

    plt.show()

plot_train_history(multi_step_history,
                   'Multi Step Training and Validation Loss')

%load_ext tensorboard
%tensorboard --logdir {log_dir}


In [ ]:
from keras.models import load_model
base_model.save('6month_changwon_model_final.h5')



In [ ]:
def create_time_steps(length):
    return list(range(-length, 0))


def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(13,6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)
  plt.plot(num_in, np.array(history[:,4]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',label='Predictied Future')
  plt.legend(loc='upper left')
  plt.show()

count =0
for x,y in val_data_multi.take(3):
#예측에 필요한 데이터 (26,5), 다음 6개월치 데이터 반환 
  multi_step_plot(x[count],y[count], base_model.predict(x)[count])
  k = x[0]
  count+=1
  #print(base_model.predict(k))
  print(x[0].shape)
  print(x.shape)
  print(x[0])
  print(y.shape)

In [ ]:
()
plt.figure(figsize=(12,6))
plt.title("Pridicting All time Housing price")
plt.plot(data['housing_price'])
pridict = []
for x,y in train_data_multi.take(1):
  print(multi_step_model.predict(x).shape)
  print(multi_step_model.predict(x))
  #for window in range(96):
  #plt.show()
  print(x.shape)
print("---------원본데이터-------------------------------") 
print(data.shape)
print(data[:,4])

#  if prediction.any():

#    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',label='Predictied Future')
#plt.legend(loc='upper left')

In [ ]:
#모델 저장
#!mkdir -p saved_model
from os import path
from google.colab import drive
notebooks_dir_name = 'notebooks'
drive.mount('/content/gdrive')
notebooks_basee_dir = path.join('./gdrive')
multi_step_model.save('/content/saved_model/large_modle')